# DIN

先实现一个 Base Model, Base Model 只是一个framework，并不能真正的运行。同时这里的 DIN 也不能运行，但是能充分体现 DIN 的思想。

In [ ]:
class BaseEmbedding(nn.Module):
    def __init__(self, fields_config):
        super(BaseEmbedding, self).__init__()
        self.user_profile_fields_config = fields_config

        self.user_profile_fields_embedding_layer = {}
        self.output_dimension = 0
        for field in self.user_profile_fields_config.keys():
            field_info = self.user_profile_fields_config[field]
            field_index = field_info['field_index']
            field_embedding_size = field_info['embedding_size']
            self.user_profile_fields_embedding_layer[field_index] = nn.Embedding(len(field_info['features']),
                                                                                 field_embedding_size)
            self.output_dimension += field_embedding_size

    def forward(self, x):
        out = torch.zeros(self.output_dimension, x.shape[1])
        idx = 0
        for field in self.fields_dict.keys():
            field_info = self.fields_dict[field]
            field_index = field_info['field_index']
            field_embedding_size = field_info['embedding_size']
            out[idx:idx + field_embedding_size, :] = self.fields_embedding[field_index](x[field_index, :])
            idx += field_embedding_size
        return out.double()


class BaseModel(nn.Module):
    def __init__(self, user_profile_fields_config, goods_fields_config, candidate_ad_fields_config,
                 context_fields_config, n=10):
        super(BaseModel, self).__init__()
        self.user_profile_feature_embedding = BaseEmbedding(user_profile_fields_config)
        self.n = 10
        self.goods_embedding = []
        for i in range(self.n):
            self.goods_embedding.append(BaseEmbedding(goods_fields_config))

        self.candidate_ad_embedding = BaseEmbedding(candidate_ad_fields_config)
        self.context_embedding = BaseEmbedding(context_fields_config)

        self.PReReLU_input_dimension = self.user_profile_feature_embedding.output_dimension + self.goods_embedding[
            0].output_dimension + self.candidate_ad_embedding.output_dimension + self.context_embedding.output_dimension +

        self.PReRelU = nn.Sequential(
            nn.Linear(self.PReReLU_input_dimension, 200, bias=True).double(),
            nn.PReLU(),
            nn.Linear(200, 80, bias=True),
            nn.PReLU(),
            nn.Softmax(2)
        )

        def forward(self, x):
            user_profile_embedding_out = self.user_profile_feature_embedding(x)
            context_embedding_out = self.context_embedding(x)
            candidate_ad_embedding_out = self.candidate_ad_embedding(x)
            good_sum_out = torch.zeros(self.goods_embedding[
                                           0].output_dimension, x.shape[1])
            for layer in self.goods_embedding:
                good_sum_out += layer(x)
            
            good_sum_out /= self.n
            out = torch.cat(
                [user_profile_embedding_out, good_sum_out, candidate_ad_embedding_out, context_embedding_out], 1)
            return self.PReRelU(out)


有了 Base Model，就可以很容易实现 DIN 了，需要在 goods embedding 的输出部分，添加 attention 模块。